In [2]:
import pandas as pd

In [12]:
!pip install memory-profiler
%load_ext memory_profiler


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached memory_profiler-0.61.0-py3-none-any.whl (31 kB)


In [16]:
df_games = pd.read_parquet('steam_games.parquet')
df_reviews = pd.read_parquet('user_reviews.parquet')
df_items = pd.read_parquet('user_items.parquet')
df_rev_games = pd.merge(df_reviews,df_games, on = "item_id", how="inner")
df_items_games = pd.merge(df_items,df_games, on = "item_id", how="inner")

In [11]:
#df_rev_games.to_parquet('reviews_and_games.parquet', engine="pyarrow")
#df_items_games.to_parquet('items_and_games.parquet', engine="pyarrow")

In [17]:
df_games.columns

Index(['app_name', 'title', 'price', 'item_id', 'developer', 'release_year',
       'Action', 'Adventure', 'Animation and Modeling', 'Audio Production',
       'Casual', 'Design and llustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing'],
      dtype='object')

In [19]:
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'posted_year', 'sentiment_analysis'], dtype='object')

In [18]:
df_items.columns

Index(['index', 'user_id', 'items_count', 'item_id', 'item_name',
       'playtime_forever'],
      dtype='object')

In [32]:
df_rev_games["posted_year"] = df_rev_games["posted_year"].astype(int)

In [14]:
df_rev_games["posted_year"].dtypes

dtype('int64')

In [ ]:
%memit lista_primos(1000)

## developer()

In [105]:
def developer(desarrollador):
    '''Devuelve la cantidad de items y porcentaje de contenido Free por año según
    la empresa desarrolladora
    
    Ejemplo de retorno:

| Año  | Cantidad de Items | Contenido Free  |
|------|-------------------|------------------|
| 2023 | 50                | 27%              |
| 2022 | 45                | 25%              |
| xxxx | xx                | xx%              |
    '''
    # Si el desarrollador no se encuentra en los dataframes:
    if desarrollador not in df_games['developer'].values:
        
        return f"ERROR: El desarrollador {desarrollador} no existe en la base de datos."   # se imprime mensaje de error
    
    # Si el desarrollador se encuentra en la base de datos:
    else:
        # Se filtra la tabla de juegos en funcion a las columnas que vamos a utilizar
        df = df_games[["item_id", "price","developer", "release_year"]]
        
        # Se filtra en el df el desarrollador ingresado
        df_developer = df[df["developer"] == desarrollador]
        
        # Se obtienen la cantidad de items totales por año:
        items_year = df_developer.groupby("release_year")["item_id"].count()  

        # Se filtra el df del desarrolladorpara aquellos juegos gratuitos (precio cero):
        df_dev_free = df_developer[df_developer["price"] == 0] 

        # Se obtiene la cantidad de items gratuitos por años
        free_items = df_dev_free.groupby("release_year")["price"].count() #cantidad de gratis por año 

        # Se calcula el porcentaje de contenido gratuito por año
        free_proportion = round((free_items / items_year) * 100, 2)  

        # Se asigna nombre a las series para poder unirlas en un dataframe:
        items_year.name = "Cantidad de Items"
        free_proportion.name = "Contenido Free"
        
        # Se unen las series en un nuevo df y se resetea index_
        df1 = pd.merge(items_year, free_proportion, on = "release_year").reset_index()
        
        # Se reemplazan los valores nulos del Dataframe por cero:
        df1 = df1.fillna(0)
        
        # Se renombra la columna "release_year":
        df1 = df1.rename(columns={"release_year" : "Año"})

        # Se da formato a la columna de contenido free:
        df1["Contenido Free"] = df1["Contenido Free"].apply(lambda x: f"{x}%")

        # Se convierte el df en diccionario
        diccionario = df1.to_dict(orient="records")     

        return diccionario

In [107]:
prueba = developer("Valve")

df_prueba = pd.DataFrame(prueba)
df_prueba.head()

,Año,Cantidad de Items,Contenido Free
0,1998,1,0.0%
1,1999,1,0.0%
2,2000,2,0.0%
3,2001,1,0.0%
4,2003,1,0.0%


## userdata()

In [157]:
def userdata(user_id):
    '''Devuelve la cantidad de dinero gastado por el usuario ingresado, el porcentaje de recomendación sobre
    las reviews realizadas y la cantidad de items

    Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, 
    "cantidad de items": 5}
    '''
    
    # Si el user_id no se encuentra en los dataframes:
    if user_id not in df_rev_games['user_id'].values:
        
        return f"ERROR: El user_id {user_id} no existe en la base de datos."   # se imprime mensaje de error
    
    # Si el user_id no se encuentra en los dataframes:
    else:
        # Se filtran las columnas del dataset a utilizar
        df_merged = df_rev_games[['user_id','item_id','price', 'recommend']]

        # Se filtran los datos en funcion al usuario especificado
        df_merged = df_merged[df_rev_games['user_id'] == user_id]

        # Se calcula la cantidad de dinero gastado por el usuario
        dinero_gastado = round(df_merged['price'].sum(), 2)

        # Se calcula la cantidad de recomendaciones del usuario
        recomendaciones = df_merged['recommend'].sum()

        # Se calcula el total de reviews del usuario
        total_reviews = df_merged.shape[0]

        # Se calcula el porcentaje de recomendaciones sobre el total de reviews   
        porcentaje_recomendacion = round(recomendaciones / total_reviews * 100, 0)

        # Se calcula la cantidad de items por usuario
        cantidad_de_items = df_merged['item_id'].nunique()

        # Crear un diccionario con los resultados
        dicc_rdos = {
        "Usuario": user_id,
        "Dinero gastado": f'{dinero_gastado} USD',
        "% de recomendación": f'{porcentaje_recomendacion}%',
        'Cantidad de items': cantidad_de_items
        }

        return dicc_rdos

In [158]:
userdata("marian")

'El user_id marian no existe en la base de datos'

In [156]:
df_rev_games[df_rev_games['user_id'] == "Nikiad"]

,user_id,item_id,recommend,posted_year,sentiment_analysis,app_name,title,price,developer,release_year,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing


## UserforGenre()

In [216]:
def UserForGenre(genero):
    '''Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de acumulación de horas
    por año
    
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
    '''
    # Si el genero no se encuentra en los dataframes:
    if genero not in df_games.columns:
        
        return f"ERROR: El género {genero} no existe en la base de datos."   # se imprime mensaje de error    
    
    # Si el genero se encuentra en los dataframes:
    else:
    # Si el genero se encuentra en los dataframes:
        # See filtra el dataframe por todos aquellos juegos catalogados dentro del género seleccionado
        df_genre = df_items_games[df_items_games[genero] == 1]

        # Se seleccionan las columnas que se van a mantener
        df_genre = df_genre[["user_id", "playtime_forever", "release_year"]]

        # Se agrupa el df por user_id sumando la cantidad de horas jugadas y buscando el usuario con el valor máximo
        user_max = df_genre.groupby("user_id")["playtime_forever"].sum().idxmax() 
        
        # Se filtra la información del usuario con más horas jugadas
        df_genre = df_genre[df_genre["user_id"] == user_max] 

        # Se agrupa la cantidad de horas jugadas por año por el usuario
        hours_year = df_genre.groupby("release_year")["playtime_forever"].sum()

        # Se agrupan las horas en un diccionario de valores
        hours_dicc = hours_year.to_dict() 

        # Se crea un diccionario vacío que almacenará los valores formateados
        hours_dicc1 = {}
                
        # Se itera sobra cada uno de los pares clave-valor del diccionario original
        for clave, valor in hours_dicc.items(): 
            key_format = f'Año: {int(clave)}'           # se da formato al año
            value_format = f'Horas: {int(valor)}'       # se da formato a la cantidad de horas jugadas
            hours_dicc1[key_format] = value_format      # se asignan los valores al diccionario creado anteriormente
            
        # Se crea la clave a utilizar en el diccionario de retorno
        clave_dicc = f'Usuario con más horas jugadas para Género {genero}'
        
        # Se retornan los valores en un diccionario: 
        return {clave_dicc : user_max, "Horas jugadas": hours_dicc1}         

In [217]:
UserForGenre("Action")

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre',
 'Horas jugadas': {'Año: 1993': 'Horas: 0',
  'Año: 1995': 'Horas: 217',
  'Año: 1996': 'Horas: 0',
  'Año: 1998': 'Horas: 0',
  'Año: 1999': 'Horas: 44',
  'Año: 2000': 'Horas: 70644',
  'Año: 2001': 'Horas: 13',
  'Año: 2002': 'Horas: 238',
  'Año: 2003': 'Horas: 7673',
  'Año: 2004': 'Horas: 127411',
  'Año: 2005': 'Horas: 21339',
  'Año: 2006': 'Horas: 896',
  'Año: 2007': 'Horas: 112784',
  'Año: 2008': 'Horas: 224',
  'Año: 2009': 'Horas: 108326',
  'Año: 2010': 'Horas: 78083',
  'Año: 2011': 'Horas: 93757',
  'Año: 2012': 'Horas: 378296',
  'Año: 2013': 'Horas: 120306',
  'Año: 2014': 'Horas: 130452',
  'Año: 2015': 'Horas: 312238',
  'Año: 2016': 'Horas: 29298',
  'Año: 2017': 'Horas: 43327'}}

In [21]:
def UserForGenre(genero):
    '''Devuelve el usuario que acumula más horas jugadas para el género dado y una lista de acumulación de horas
    por año
    
    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
    '''
    # See filtra el dataframe por todos aquellos juegos catalogados dentro del género seleccionado
    df_filter = df_games[df_games[genero] == 1]
    
    # Se seleccionan las columnas necesarias de los dataframes:
    df_genre = pd.merge(df_filter[['item_id','release_year']], df_items[['item_id',"user_id", "playtime_forever"]], on="item_id", how = 'inner')

    # Se agrupa el df por user_id sumando la cantidad de horas jugadas y buscando el usuario con el valor máximo
    user_max = df_genre.groupby("user_id")["playtime_forever"].sum().idxmax() 
    
    # Se filtra la información del usuario con más horas jugadas
    df_genre = df_genre[df_genre["user_id"] == user_max] 

    # Se agrupa la cantidad de horas jugadas por año por el usuario
    hours_year = df_genre.groupby("release_year")["playtime_forever"].sum()

    # Se agrupan las horas en un diccionario de valores
    hours_dicc = hours_year.to_dict() 

    # Se crea un diccionario vacío que almacenará los valores formateados
    hours_dicc1 = {}
            
    # Se itera sobra cada uno de los pares clave-valor del diccionario original
    for clave, valor in hours_dicc.items(): 
        key_format = f'Año: {int(clave)}'           # se da formato al año
        value_format = f'Horas: {int(valor)}'       # se da formato a la cantidad de horas jugadas
        hours_dicc1[key_format] = value_format      # se asignan los valores al diccionario creado anteriormente

    # Se crea la clave a utilizar en el diccionario de retorno
    clave_dicc = f'Usuario con más horas jugadas para Género {genero}'
    
    # Se retornan los valores en un diccionario: 
    return {clave_dicc : user_max, "Horas jugadas": hours_dicc1}

In [24]:
df_games[df_games["Action"] == 1]

,app_name,title,price,item_id,developer,release_year,Action,Adventure,Animation and Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
3,弹炸人2222,弹炸人2222,0.99,767400,彼岸领域,2017,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Battle Royale Trainer,Battle Royale Trainer,3.99,772540,Trickjump Games Ltd,2018,1,1,0,0,...,0,0,0,1,0,0,0,0,0,0
7,Army of Tentacles: (Not) A Cthulhu Dating Sim:...,Army of Tentacles: (Not) A Cthulhu Dating Sim:...,10.99,770380,Stegalosaurus Game Development,2018,1,1,0,0,...,0,1,0,0,0,0,0,0,0,0
14,The Warrior Of Treasures,The Warrior Of Treasures,0.99,768060,Tero Lunkka,2018,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22513,Chicken Shoot Gold,Chicken Shoot Gold,6.99,259340,ToonTRAXX Studios,2003,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22514,Day of Defeat,Day of Defeat,4.99,30,Valve,2003,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22516,Unreal Tournament 2004: Editor's Choice Edition,Unreal Tournament 2004: Editor's Choice Edition,14.99,13230,"Epic Games, Inc.",2004,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22517,Counter-Strike: Condition Zero,Counter-Strike: Condition Zero,9.99,80,Valve,2004,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
UserForGenre("Action")

KeyError: ('item_id', 'release_year')

## best_developer_year()

In [36]:
def top_developer_year(año):
    '''
    Devuelve el top 3 de desarrolladores con juegos más recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    '''
    # Se seleccionan las columnas a utilizar
    df_year = df_rev_games[['posted_year','app_name','recommend', 'sentiment_analysis', 'developer']]

    # Se filtran los datos por el año ingresado:
    df_year = df_year[df_year['posted_year'] == año]

    # Se filtran las recomendaciones de usuarios:
    df_year = df_year[(df_year['recommend'] == 1) & (df_year['sentiment_analysis'] == 2)]

    # Se cuentan las recomendaciones para cada desarrollador
    recomendaciones_developer = df_year.groupby('developer')["app_name"].count()

    # Se ordenan las recomendaciones por orden descendente, se seleccionan las primeras 3 y se convierten a lista:
    best_developers = recomendaciones_developer.sort_values(ascending=False).head(3).index.to_list()

    # Se devuelven los resultados en una lista
    return {"Puesto 1" : best_developers[0], "Puesto 2" : best_developers[1], "Puesto 3" : best_developers[2]}
    
def best_developer_year(año):
    '''
    Devuelve el top 3 de desarrolladores con juegos más recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    '''
    try:
        año_int = int(año)
        
        return top_developer_year(año_int)
    
    except Exception as e:
        
        return {"error": str(e)}

In [38]:
best_developer_year(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC',
 'Puesto 3': 'Blue Mammoth Games'}

In [34]:
# Opcion 2:
def validar_entero(year):
    """ Valida que el año provisto sea entero """

    try:
        return int(year)
    except:
        return False
    
def validar_anio(year):    
    """ Valida que el año provisto esté dentro del dataset """

    min_year = int(df_rev_games['release_year'].min())
    max_year = int(df_rev_games['posted_year'].max())

    if min_year > int(year) or int(year) > max_year:
        return min_year, max_year
    return False

def best_developer_year2(año):
    '''
    Devuelve el top 3 de desarrolladores con juegos más recomendados por usuarios para el año dado.
    (reviews.recommend = True y comentarios positivos)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
    '''
    
    # Validamos que sea entero
    if not validar_entero(año):
        return {"error" : f"{año} no es un número entero válido"}
    
    #Convertimos a entero
    año = int(año)

    # Validamos el año esté dentro del rango
    if result := validar_anio(año):
        return {"error" : f"El año de lanzamiento no forma parte del dataset. Por favor pruebe valores entre {result[0]} y {result[1]}"}
    
    # Se seleccionan las columnas a utilizar
    df_year = df_rev_games[['posted_year','app_name','recommend', 'sentiment_analysis', 'developer']]

    # Se filtran los datos por el año ingresado:
    df_year = df_year[df_year['posted_year'] == año]

    # Se filtran las recomendaciones de usuarios:
    df_year = df_year[(df_year['recommend'] == 1) & (df_year['sentiment_analysis'] == 2)]

    # Se cuentan las recomendaciones para cada desarrollador
    recomendaciones_developer = df_year.groupby('developer')["app_name"].count()

    # Se ordenan las recomendaciones por orden descendente, se seleccionan las primeras 3 y se convierten a lista:
    best_developers = recomendaciones_developer.sort_values(ascending=False).head(3).index.to_list()

    # Se devuelven los resultados en una lista
    return {"Puesto 1" : best_developers[0], "Puesto 2" : best_developers[1], "Puesto 3" : best_developers[2]}

In [35]:
best_developer_year1(2017)

{'Puesto 1': 'Smartly Dressed Games',
 'Puesto 2': 'Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC',
 'Puesto 3': 'Blue Mammoth Games'}

## developer_reviews_analysis()

In [102]:
def developer_reviews_analysis(desarrollador:str):
    '''Devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de 
    reseñas positivas y negativas de usuarios
    
    Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}
    '''
    # Si el desarrollador no se encuentra en los dataframes:
    if desarrollador not in df_games['developer'].values:
        
        return f"ERROR: El desarrollador {desarrollador} no existe en la base de datos."    # se imprime mensaje de error
    
    # Si el desarrollador se encuentra en la base de datos:
    else:
        # Se filtran las columnas a utilizar y se eliminan duplicados
        df_merged = df_rev_games[['developer','sentiment_analysis']]
        
        # Se filtran los datos por el developer ingresado
        df_merged = df_merged[df_merged["developer"] == desarrollador]

        # Se obtienen la cantidad de reviews positivas y negativas
        positive_reviews = df_merged[df_merged["sentiment_analysis"] == 2].shape[0]
        negative_reviews = df_merged[df_merged["sentiment_analysis"] == 0].shape[0]
        
        # Se juntan los valores en un f-string
        resumen_reviews = f"[Negative = {negative_reviews}, Positive = {positive_reviews}]"
        
        # Se almacenan los resultados en un diccionario
        dicc = {desarrollador: resumen_reviews}

        # Se devuelve un diccionario con los resultados obtenidos
        return dicc 

In [103]:
developer_reviews_analysis("Valve")

{'Valve': '[Negative = 1080, Positive = 5988]'}

In [ ]:
top_desarrolladores = top_desarrolladores_recomendados(year)

    # Devolver el resultado
    return {"top_desarrolladore= 1" : top_desarrolladores[0], "top_desarrollador 2" : top_desarrolladores[1], "top_desarrollador 3" : top_desarrolladores[2]}